In [1]:
import sys
import math
import pandas as pd
import geopandas as gpd

# import custom modules
sys.path.append('../../utils/')
import data_paths
from traffic_counts import TrafficCounts

In [2]:
count = TrafficCounts()


AttributeError: 'TrafficCounts' object has no attribute 'dtv'

In [4]:
count.get_dtv()

AttributeError: 'Series' object has no attribute 'dtv_KFZ'

In [ ]:
count.get_daily_Scaling_factor()